# Format the CF data into the SWDE
website = domain

In [1]:
import pandas as pd
import pandavro as pdx
from ast import literal_eval
from pathlib import Path
import os   
import sys
import shutil
from tqdm import tqdm
from markuplmft.fine_tuning.run_swde.prepare_data import get_dom_tree

# Load

## Full Data

In [2]:
# data_path = "../../../web-annotation-extractor/data/processed/develop/dataset.avro"
# df = pdx.read_avro(data_path)
# df.annotations = df.annotations.apply(literal_eval)
# len(df)

## Positive Data (containing at least one annotation)

In [3]:
## Create a sample that contains only positives:
# df['annotations_len'] = df['annotations'].apply(len)
# df_sample = df[df['annotations_len'] > 0]
# save_path = data_path.replace('.avro', f'_positives({len(df_sample)}).pkl')
# print(save_path)
# df_sample.to_pickle(save_path)

In [4]:
data_path = "../../../web-annotation-extractor/data/processed/train/dataset_positives(11641).pkl"
df = pd.read_pickle(data_path)
len(df)

11641

# Select Tags

In [5]:
annotation_tags = ["PAST_CLIENT", "OFFICE_LOCATION", "CASE_STUDY"][:1]
annotation_tags

['PAST_CLIENT']

# Clean Data

## Remove hiphen from domains

In [6]:
df.domain = df.domain.apply(lambda x: x.replace('-', ''))

## Assert that domain don't contain parenthesis


In [7]:
assert len(df[df['domain'].apply(lambda x: '(' in x or ')' in x)]) == 0
# Remove parenthesis from domain in case there is (assertion above fails)
# ... df = df[~df['domain'].apply(lambda x: '(' in x or ')' in x)]

# Get text annotations per tag

In [8]:
for tag in annotation_tags:
    print(f'- {tag}:')
    df[f'annotations-{tag}'] = df['annotations'].apply(lambda x: x.get(tag))        
    df[f'text-{tag}'] = df.dropna(subset=[f'annotations-{tag}'])[f'annotations-{tag}'].apply(lambda x: [y['text'] for y in x])        

- PAST_CLIENT:


# Remove samples that don't have html

In [9]:
annotations_without_html = len([y for x in df[df['html'] == 'PLACEHOLDER_HTML'].dropna(subset=['annotations-PAST_CLIENT'])['text-PAST_CLIENT'] for y in x])
annotations_without_html

575

In [10]:
df = df[df['html'] != 'PLACEHOLDER_HTML']

# Remove annotations that cannot be found in the xpaths of the html

In [12]:
initial_amount_of_label = len([y for x in df.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])
print(f"Initial amount of labels: {initial_amount_of_label}")

all_new_annotations = []

for i, row in tqdm(df.iterrows()):
    if not row.isnull()[f'text-{tag}']:
        clean_dom_tree = get_dom_tree(row['html'], 'website')
        
        annotations_that_can_be_found = []
        annotations_that_cannot_be_found = []
        for text_annotation in row[f'text-{tag}']:
            for node in clean_dom_tree.iter():
                if node.text:
                    if text_annotation in node.text:
                        annotations_that_can_be_found.append(text_annotation)
                        break
                if node.tail:
                    if text_annotation in node.tail:
                        annotations_that_can_be_found.append(text_annotation)
                        break
                # for html_tag, xpath_content in node.items():
                #     if text_annotation in xpath_content:
                #         annotations_that_can_be_found.append(text_annotation)
                #         break
            annotations_that_cannot_be_found.append(text_annotation)
            
        if len(annotations_that_cannot_be_found) > 0:
            print(f"Cannot be found in {i} \t: {annotations_that_cannot_be_found}")
            print()

        all_new_annotations.append(annotations_that_can_be_found)
    else:
        all_new_annotations.append(None)
        
df[f'text-{tag}'] = all_new_annotations

final_amount_of_label = len([y for x in df.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])
print(f"Final amount of labels: {final_amount_of_label}")
print(f"Number of labels lost because they couldn't be found in the page: {initial_amount_of_label - final_amount_of_label}")

Initial amount of labels: 12085


11it [00:00, 79.86it/s]

Cannot be found 4 	: ['Northern Tool & Equipment']

Cannot be found 8 	: ['Cumberland Valley National Bank']

Cannot be found 10 	: ['Sally’s Beauty']

Cannot be found 57 	: ['https://go.applause.com/rs/539-ckp-074/images/walmart.svg', 'https://go.applause.com/rs/539-ckp-074/images/petco.svg', 'https://go.applause.com/rs/539-ckp-074/images/logo-ford-dxc-2018.svg', 'https://go.applause.com/rs/539-ckp-074/images/blue-apron-logo.svg']

Cannot be found 58 	: ['Ubisoft']

Cannot be found 59 	: ['AT&T']

Cannot be found 60 	: ['Bank of America']

Cannot be found 61 	: ['Copper State Communication']



19it [00:02,  7.52it/s]

Cannot be found 62 	: ['Meyer Corporation']

Cannot be found 63 	: ["SOS Children's Villages"]

Cannot be found 64 	: ['Standard Life']

Cannot be found 65 	: ['Telarus']



...

Cannot be found 11602 	: ['https://mou7z1xm2q3rzzg717wtqleu-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/customer-logo-academy.png', 'https://mou7z1xm2q3rzzg717wtqleu-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/customer-logo-fifth-third-bank.png', 'https://mou7z1xm2q3rzzg717wtqleu-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/customer-logo-finance-of-america.png', 'https://mou7z1xm2q3rzzg717wtqleu-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/customer-logo-cherry-creek-mortgage.png', 'https://mou7z1xm2q3rzzg717wtqleu-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/customer-logo-cross-country-mortgage.png', 'https://mou7z1xm2q3rzzg717wtqleu-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/customer-logo-diamond-residential-mortgage.png', 'https://mou7z1xm2q3rzzg717wtqleu-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/customer-logo-churchill-mortgage.png', 'https://mou7z1xm2q3rzzg717wtqleu-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/customer-logo-fir

9472it [03:55, 60.57it/s]

Cannot be found 11612 	: ['https://yesler.com/wp-content/uploads/2019/05/ysm-275-logos-varwidth-amazon.png', 'https://yesler.com/wp-content/uploads/2019/12/ysm-275-claris-logo-dec-2019.png', 'https://yesler.com/wp-content/uploads/2019/05/ysm-275-logos-0519_commvault.png', 'https://yesler.com/wp-content/uploads/2019/05/ysm-275-logos-varwidth-dataiku.png', 'https://yesler.com/wp-content/uploads/2019/05/ysm-275-logos-varwidth-enviance.png', 'https://yesler.com/wp-content/uploads/2019/05/ysm-275-logos-varwidth-google.png', 'https://yesler.com/wp-content/uploads/2019/12/ysm-275-ingram-logo-dec-2019.png', 'https://yesler.com/wp-content/uploads/2019/12/ysm-275-magento-logo-dec-2019.png', 'https://yesler.com/wp-content/uploads/2019/05/ysm-275-logos-varwidth-marketo.png', 'https://yesler.com/wp-content/uploads/2019/05/ysm-275-logos-varwidth-ptc.png', 'https://yesler.com/wp-content/uploads/2020/07/ysm-275-logos-varwidth-new-riverbed.png', 'https://yesler.com/wp-content/uploads/2019/05/ysm-275-lo

9476it [03:56, 40.15it/s]


Cannot be found 11637 	: ['AC Milan']

Cannot be found 11638 	: ['The Civil Aviation Authority']

Cannot be found 11639 	: ['Ivy League']

Final amount of labels: 9644
Number of labels lost because they couldn't be found in the page: 2441


In [13]:
len(df)

9476

In [14]:
print("With text and tail the annotation coverage is:", 9644/12085)

With text and tail the annotation coverage is: 0.7980140670252379


In [15]:
print("With text, tail and xpath_content the annotation coverage is:", 11500/12085)

With text, tail and xpath_content the annotation coverage is: 0.9515928837401738


## Debug (find why annotation could not be found in html):

In [16]:
# row = df.iloc[3143]
# print(row)
# clean_dom_tree = get_dom_tree(row['html'], 'website')
# print()
# for node in clean_dom_tree.iter():
#     for html_tag, xpath_content in node.items():
#         print('text:', node.text)
#         print('tail:', node.tail)
#         print('xpath:', xpath_content)        

In [17]:
# html = row['html']

# Remove image link annotations

In [20]:
df[f'text-{tag}'] = df[f'text-{tag}'].dropna().apply(lambda annotations: [annotation  for annotation in annotations if 'http' not in annotation])

# Remove samples without annotation

In [22]:
df = df[df[f'text-{tag}'].fillna('').apply(list).apply(len) > 0]
df

,url,domain,content_type,annotations,html,crawl_index,source,source_excel,source_batch,annotations_len,annotations-PAST_CLIENT,text-PAST_CLIENT
4,https://www.3sisecurity.com/case-study-norther...,3sisecurity.com,text/html,{'CASE_STUDY': [{'text': 'there are unique hur...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-31-...,cloudfactory,['Batch - 2021-03-12 (S-B7)/Templates/3SI SECU...,"['Batch - 2021-03-12 (S-B7)', 'Batch - 2021-05...",2,"[{'text': 'Northern Tool & Equipment', 'value'...",[Northern Tool & Equipment]
8,https://www.3sisecurity.com/check-out-what-cum...,3sisecurity.com,text/html,{'PAST_CLIENT': [{'text': 'Cumberland Valley N...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-31-...,cloudfactory,Batch - 2021-03-12 (S-B7)/Templates/3SI SECURI...,Batch - 2021-03-12 (S-B7),1,"[{'text': 'Cumberland Valley National Bank', '...",[Cumberland Valley National Bank]
10,https://www.3sisecurity.com/why-use-gps-becaus...,3sisecurity.com,text/html,{'CASE_STUDY': [{'text': 'organized retail cri...,"<!DOCTYPE html>\n<!--[if IE 6]>\n<html id=""ie6...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,['Batch - 2021-03-12 (S-B7)/Templates/3SI SECU...,"['Batch - 2021-03-12 (S-B7)', 'Batch - 2021-05...",2,"[{'text': 'Sally’s Beauty', 'value': None}]",[Sally’s Beauty]
58,https://resources.aprimo.com/success-stories/-...,aprimo.com,text/html,"{'PAST_CLIENT': [{'text': 'Ubisoft', 'value': ...","<!DOCTYPE html>\n<html itemscope itemtype=""htt...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-03-12 (S-B8)/Templates/To be uplo...,Batch - 2021-03-12 (S-B8),1,"[{'text': 'Ubisoft', 'value': 'http://graph.gl...",[Ubisoft]
59,https://resources.aprimo.com/success-stories/a...,aprimo.com,text/html,"{'PAST_CLIENT': [{'text': 'AT&T', 'value': 'ht...","<!DOCTYPE html>\n<html itemscope itemtype=""htt...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-03-12 (S-B8)/Templates/To be uplo...,Batch - 2021-03-12 (S-B8),1,"[{'text': 'AT&T', 'value': 'http://graph.globa...",[AT&T]
...,...,...,...,...,...,...,...,...,...,...,...,...
11631,https://yesler.com/b2b-marketing-case-studies/...,yesler.com,text/html,{'CASE_STUDY': [{'text': 'when product marketi...,<!doctype html>\n<!--[if lt IE 7]> <html class...,s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-06-17 (MK1)/Templates/Yesler.xlsx,Batch - 2021-06-17 (MK1),2,"[{'text': 'Microsoft', 'value': 'http://graph....",[Microsoft]
11632,https://yesler.com/b2b-marketing-case-studies/...,yesler.com,text/html,{'CASE_STUDY': [{'text': 'hey did you know you...,<!doctype html>\n<!--[if lt IE 7]> <html class...,s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-06-17 (MK1)/Templates/Yesler.xlsx,Batch - 2021-06-17 (MK1),2,"[{'text': 'Zillow', 'value': 'http://graph.glo...",[Zillow]
11637,https://www.zerofox.com/resources/ac-milan-cas...,zerofox.com,text/html,"{'PAST_CLIENT': [{'text': 'AC Milan', 'value':...","<!doctype html>\n<html class=""no-js "" lang=""en...",s3://globality-raw/crawl-indexes/full-upload-j...,cloudfactory,Batch - 2021-03-12 (S-B3)/Templates/ZEROFOX IN...,Batch - 2021-03-12 (S-B3),1,"[{'text': 'AC Milan', 'value': None}]",[AC Milan]
11638,https://www.zerofox.com/resources/civil-aviati...,zerofox.com,text/html,{'PAST_CLIENT': [{'text': 'The Civil Aviation ...,"<!doctype html>\n<html class=""no-js "" lang=""en...",s3://globality-raw/crawl-indexes/crawl-aug-6-2...,cloudfactory,Batch - 2021-03-12 (S-B3)/Templates/ZEROFOX IN...,Batch - 2021-03-12 (S-B3),1,"[{'text': 'The Civil Aviation Authority', 'val...",[The Civil Aviation Authority]


# Number of Domains

In [23]:
df.domain.value_counts()

hitt.com                  229
blackbaud.com             168
futurebrand.com           144
tableau.com               143
fastsigns.com             106
                         ... 
environmentsatwork.com      1
equinix.nl                  1
opswat.com                  1
opentext.com                1
progress.com                1
Name: domain, Length: 286, dtype: int64

# Format and Save data

In [24]:
raw_data_folder = Path.cwd().parents[2] / 'swde/my_CF_sourceCode'

if os.path.exists(raw_data_folder):
    print(f'Are you sure you want to remove this folder? (y/n) \n{raw_data_folder}')
    answer = input()    
    if answer == 'y':
        try:
            shutil.rmtree(raw_data_folder)
            print(f"REMOVED: {raw_data_folder}")
        except OSError as e:
            print ("Error: %s - %s." % (e.filename, e.strerror))
else:
    print(f"File '{groundtruth_data_path}' not found in the directory")
    
domains = list(df.domain.value_counts().index)

groundtruth_data_path = raw_data_folder / 'groundtruth' / 'WAE'   
groundtruth_data_path.mkdir(parents=True, exist_ok=True)

for e, domain in enumerate(domains):        
    df_domain = df[df.domain == domain]
    
    print(f"{e:>3}: {len(df_domain):>5} page(s) - {domain:>25}")
    
    domain_annotations = {}    
    
    page_count = 0
    domain_len = len(df_domain)
    
    for enum, df_page in df_domain.iterrows():
        # Save html
        html = df_page['html']
        raw_data_path = raw_data_folder / 'WAE' / f"WAE-{domain}({domain_len})"
        raw_data_path.mkdir(parents=True, exist_ok=True)
        raw_data_path = (raw_data_path / str(page_count).zfill(4)).with_suffix('.htm')
        print(raw_data_path)
        
        Html_file = open(raw_data_path, "w")
        Html_file.write(html)
        Html_file.close()
        
        page_count += 1
                
        # Get groundtruth for page for each tag
        for tag in annotation_tags:            
            domain_annotations[tag] = domain_annotations.get(tag, [])
            if not df_page.isnull()[f'text-{tag}']:
                annotations = df_page[f'text-{tag}']                
                # Remove image links from text annotation
                annotate = [annotation.strip() if (annotation and 'http' not in annotation.strip()) else '' for annotation in annotations]
            else:
                annotate = []
            # print(f'annotate: \n{annotate} - {len(annotate)}')            
            domain_annotations[tag].append(annotate)            
        print()
        # if raw_data_path.name == '0042.htm':
        #     break
        
    # Save groundtruth    
    for tag, page_annotations in domain_annotations.items():
        groundtruth_data_tag_path = groundtruth_data_path / f"WAE-{domain}-{tag}.txt"
        print(groundtruth_data_tag_path)

        page_annotations_df = pd.DataFrame(page_annotations)
        
        # Count number of annotations
        page_annotations_df['number of values'] = page_annotations_df.T.count()        
        
        # Invert columns order 
        cols = page_annotations_df.columns.tolist()
        page_annotations_df = page_annotations_df[cols[::-1]] 
        
        # Get page index
        page_annotations_df.reset_index(inplace=True)
        page_annotations_df['index'] = page_annotations_df['index'].apply(lambda x: str(x).zfill(4))
        
        # Add one extra row on the top
        page_annotations_df.loc[-1] = page_annotations_df.count()  # adding a row
        page_annotations_df.index = page_annotations_df.index + 1  # shifting index
        page_annotations_df = page_annotations_df.sort_index()
        
        page_annotations_df.to_csv(groundtruth_data_tag_path, sep="\t", index=False)

Are you sure you want to remove this folder? (y/n) 
/data/GIT/swde/my_CF_sourceCode


 y


REMOVED: /data/GIT/swde/my_CF_sourceCode
  0:   229 page(s) -                  hitt.com
/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0000.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0001.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0002.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0003.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0004.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0005.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0006.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0007.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0008.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0009.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0010.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0011.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0012.htm

/data/GIT/swde/my_CF_sourceCode/WAE/WAE-hitt.com(229)/0013.htm

/data/GIT/swde/m

---